In [ ]:
idir = "/Users/BlacKay/Documents/Projects/Images/test/bust/"
idir = "/Users/BlacKay/Documents/Projects/Images/test/templeRing/"
import cv2

nimages = 10
images = []

for i in range(nimages):
    iname = f"{idir}/{i:08d}.jpg"
    img = cv2.imread(iname)
    images.append(img)





In [ ]:

import cv2
import numpy as np
# Initialize SIFT (parameters roughly matching your C++ example)
# Note: Not all SiftParams map directly to cv2.SIFT_create(), so we use similar ones.
sift = cv2.SIFT_create(
    nfeatures=0,           # similar to octave settings
    nOctaveLayers=3,       # 2nd param (matches your 3)
    contrastThreshold=0.04,
    edgeThreshold=10,
    sigma=1.6
)

keys = [[] for _ in range(nimages)]
descs = [[] for _ in range(nimages)]

for i in range(nimages):
    gray = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
    kpts, descriptors = sift.detectAndCompute(gray, None)
    keys[i] = kpts
    descs[i] = descriptors

In [ ]:
from typing import Tuple
import numpy as np

class DMatch:
    def __init__(self, ikey1=None, ikey2=None, pt1=None, pt2=None, dist=None):
        # Handle different constructor variants
        if pt1 is not None and pt2 is not None:
            self.ind_key_ = (ikey1, ikey2)
            self.point_ = (np.array(pt1, dtype=float), np.array(pt2, dtype=float))
            self.dist_ = float(dist)
        elif ikey1 is not None and ikey2 is not None and dist is not None:
            self.ind_key_ = (ikey1, ikey2)
            self.point_ = (None, None)
            self.dist_ = float(dist)
        elif ikey1 is None and ikey2 is None and dist is None:
            self.ind_key_ = (None, None)
            self.point_ = (None, None)
            self.dist_ = None
        elif isinstance(ikey1, DMatch):
            # Copy constructor
            match = ikey1
            self.ind_key_ = match.ind_key_
            self.point_ = match.point_
            self.dist_ = match.dist_
        else:
            raise ValueError("Invalid constructor arguments for DMatch")

    def dist(self) -> float:
        """Return the match distance."""
        return self.dist_


In [ ]:
import numpy as np
from typing import List, Tuple

class Pair:
    def __init__(self, cam1=None, cam2=None, matches=None):
        """
        Python equivalent of the C++ Pair constructors.
        Supports:
          - Pair()
          - Pair(cam1, cam2)
          - Pair(cam1, cam2, matches)
        """
        self.cams_ = []
        self.matches_ = []
        self.F_ = np.eye(3, dtype=float)
        self.E_ = np.eye(3, dtype=float)
        self.intrinsics_mat_ = []
        self.extrinsics_mat_ = []

        if cam1 is not None and cam2 is not None:
            self.init(cam1, cam2)
            if matches is not None:
                # Matches can be list of (Vec2f, Vec2f) or list of DMatch
                if all(isinstance(m, DMatch) for m in matches):
                    self.matches_ = matches
                elif all(isinstance(m, tuple) and len(m) == 2 for m in matches):
                    # Convert raw (pt1, pt2) to DMatch objects with dummy distance
                    self.matches_ = [DMatch(i, i, m[0], m[1], 0.0) for i, m in enumerate(matches)]
                else:
                    raise ValueError("matches must be list of DMatch or list of (Vec2f, Vec2f) pairs")

    def init(self, ind_cam1: int, ind_cam2: int):
        """Initialize the pair with two camera indices."""
        self.cams_ = [ind_cam1, ind_cam2]

    def update_matches(self, vote_inlier: np.ndarray):
        """Update matches using an inlier mask (1 for inlier, 0 for outlier)."""
        self.matches_ = [m for m, v in zip(self.matches_, vote_inlier) if v]

    def update_intrinsics(self, f: float, w: int, h: int):
        """Compute intrinsics matrix for both cameras based on focal length and image size."""
        K = np.array([[f, 0, w / 2],
                      [0, f, h / 2],
                      [0, 0, 1]], dtype=float)
        self.intrinsics_mat_ = [K.copy(), K.copy()]

    def __lt__(self, rhs: 'Pair') -> bool:
        """Implements operator< for sorting."""
        return tuple(self.cams_) < tuple(rhs.cams_)

    def baseline_angle(self) -> float:
        """Placeholder for baseline angle computation."""
        # You can compute this later from extrinsics if available
        return 0.0  # Placeholder


In [ ]:
import cv2
import numpy as np
import os

pairs = []
odir = "/path/to/output/"  # ← set this properly

# Equivalent of: Matcher_Param(0.6*0.6, 50)
ratio_thresh = 0.6 * 0.6
max_matches = 50

# FLANN parameters for SIFT descriptors (float32)
FLANN_INDEX_KDTREE = 1
flann_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
matcher = cv2.FlannBasedMatcher(flann_params, search_params)

matches_pairwise = [[] for _ in range(nimages - 1)]

for i in range(nimages - 1):
    matches_pairwise[i] = [[] for _ in range(nimages - (i + 1))]
    for j in range(i + 1, nimages):
        # Perform KNN matching (k=2) for ratio test
        knn_matches = matcher.knnMatch(descs[i], descs[j], k=2)

        # Lowe’s ratio test
        good_matches = []
        for m, n in knn_matches:
            if m.distance < ratio_thresh * n.distance:
                good_matches.append(DMatch(m.queryIdx, m.trainIdx, m.distance))

        # Keep only top matches if needed
        good_matches = sorted(good_matches, key=lambda x: x.dist_)[:max_matches]

        matches_pairwise[i][j - (i + 1)] = good_matches

        # Store the pair
        pairs.append(Pair(i, j, good_matches))

        # Write matches to text file
        fname = os.path.join(odir, f"matching{i+1}_{j+1}.txt")
        with open(fname, "w") as f:
            for m in good_matches:
                f.write(f"{m.ind_key_[0]} {m.ind_key_[1]} {m.dist_}\n")

        # (Optional) draw matches
        if False:
            img_matches = cv2.drawMatches(
                images[i], keys[i],
                images[j], keys[j],
                [cv2.DMatch(m.ind_key_[0], m.ind_key_[1], m.dist_) for m in good_matches],
                None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )
            out_path = os.path.join(odir, f"matching{i+1}_{j+1}.jpg")
            cv2.imwrite(out_path, img_matches)

        print(f"Image ({i+1}, {j+1}): matches number: {len(good_matches)}")


In [ ]:
import numpy as np
import cv2

thresh_inlier_ratio = 0.3  # example threshold, adjust as in your C++ code

for pair in pairs:
    nmatches = len(pair.matches_)
    ind1, ind2 = pair.cams_

    print("*******************************")
    print(f" 2-View SfM of image {ind1+1} and {ind2+1}")
    print("*******************************")

    # --- Build matched points from DMatch objects ---
    pts1 = []
    pts2 = []
    for m in pair.matches_:
        # Retrieve keypoint coordinates from the original keypoint lists
        pt1 = keys[ind1][m.ind_key_[0]].pt
        pt2 = keys[ind2][m.ind_key_[1]].pt
        pts1.append(pt1)
        pts2.append(pt2)

    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)

    if len(pts1) < 8:
        print("Not enough matches for fundamental matrix estimation, skipping.")
        continue

    # --- Estimate Fundamental Matrix with RANSAC ---
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, 1e-8, 0.99)

    if F is None or mask is None:
        print("Fundamental matrix estimation failed.")
        continue

    inlier_ratio = float(np.sum(mask)) / len(mask)
    print(f"Ratio of matching inliers: {inlier_ratio:.3f}")

    if inlier_ratio < thresh_inlier_ratio:
        continue

    # Store F in the Pair object (ensure it’s 3x3)
    pair.F_ = F.reshape(3, 3)

    # Optional: store inlier matches
    inlier_mask = mask.ravel().astype(bool)
    pair.update_matches(inlier_mask)

    # (other code continues here, e.g. Essential matrix estimation, triangulation, etc.)


In [ ]:
import numpy as np
import cv2

f = 1520.4
w, h = int(302.32 * 2), int(246.87 * 2)

# Update intrinsics (fills pair.intrinsics_mat_ with two K matrices)
pair.update_intrinsics(f, w, h)

# Compute Essential matrix: E = K2ᵀ * F * K1
K1 = pair.intrinsics_mat_[0]
K2 = pair.intrinsics_mat_[1]
pair.E_ = K2.T @ pair.F_ @ K1

# ---- Decompose Essential Matrix to get R and t ----
def Rt_from_E(pair):
    E = pair.E_

    # Recover pose from essential matrix
    pts1 = []
    pts2 = []
    for m in pair.matches_:
        pt1 = keys[pair.cams_[0]][m.ind_key_[0]].pt
        pt2 = keys[pair.cams_[1]][m.ind_key_[1]].pt
        pts1.append(pt1)
        pts2.append(pt2)

    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)

    # Use OpenCV's recoverPose
    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, K1)

    # Store extrinsics for both cameras (assuming first camera is at origin)
    M1 = np.hstack((np.eye(3), np.zeros((3, 1))))  # [R|t] = [I|0]
    M2 = np.hstack((R, t))
    pair.extrinsics_mat_ = [M1, M2]

    print("Recovered rotation:\n", R)
    print("Recovered translation:\n", t)

# Call it
Rt_from_E(pair)


In [ ]:
import numpy as np
from typing import List

class Graph:
    def __init__(self, pair=None):
        """
        Equivalent to:
          Graph()          → empty graph
          Graph(const Pair&) → initialize from a Pair
        """
        self.ncams_ = 0
        self.ind_cam_ = []
        self.intrinsics_mat_ = []
        self.extrinsics_mat_ = []
        self.tracks_ = []
        self.structure_points_ = []

        if pair is not None:
            self.init(pair)

    def init(self, pair):
        """Initialize graph from a Pair object."""
        self.ncams_ = len(pair.cams_)
        self.ind_cam_ = pair.cams_.copy()
        self.intrinsics_mat_ = pair.intrinsics_mat_.copy()
        self.extrinsics_mat_ = pair.extrinsics_mat_.copy()

    def index(self, icam: int) -> int:
        """
        Return the index of a camera id in ind_cam_.
        Returns -1 if not found.
        """
        try:
            return self.ind_cam_.index(icam)
        except ValueError:
            return -1

    def size(self) -> int:
        """Return the number of cameras."""
        return self.ncams_

    @staticmethod
    def intersect(tracks1: List["Track"], tracks2: List["Track"]) -> List[int]:
        """
        Compute intersection between two track sets.
        (Assuming each Track has an attribute 'id' or similar identifier)
        """
        ids1 = {t.id for t in tracks1}
        ids2 = {t.id for t in tracks2}
        return list(ids1 & ids2)
    
    def add_track(self, track):
        self.tracks_.append(track)

    def add_struct_pt(self, struct_pt):
        self.structure_points_.append(struct_pt)

    @staticmethod
    def merge_tracks(track1, track2, feats_common):
        # merge observations from track2 into track1
        for (cam_id, _) in feats_common:
            if cam_id not in [obs.cam_id for obs in track1.observations]:
                track1.observations.append(next(o for o in track2.observations if o.cam_id == cam_id))


In [ ]:
import numpy as np
import cv2

def triangulate_nonlinear(graph):
    """
    Triangulate 3D points between the first two cameras in the graph.
    Stores them into graph.structure_points_.
    """
    if len(graph.extrinsics_mat_) < 2:
        print("Need at least two camera poses to triangulate.")
        return

    P1 = graph.intrinsics_mat_[0] @ graph.extrinsics_mat_[0]
    P2 = graph.intrinsics_mat_[1] @ graph.extrinsics_mat_[1]

    # Collect corresponding points from tracks (assuming you have them)
    pts1, pts2 = [], []
    for track in graph.tracks_:
        # Assuming each Track has image points (pt1, pt2)
        pts1.append(track.pt1)
        pts2.append(track.pt2)

    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)

    if len(pts1) == 0:
        print("No tracks to triangulate.")
        return

    # Linear triangulation
    points4D_h = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)

    # Convert from homogeneous to 3D
    points3D = (points4D_h[:3] / points4D_h[3]).T

    # Store the structure points in the graph
    graph.structure_points_ = points3D


def reprojection_error(graph):
    """
    Compute average reprojection error for all 3D points in the graph.
    """
    if not hasattr(graph, "structure_points_") or len(graph.structure_points_) == 0:
        return np.inf

    total_error = 0.0
    total_points = 0

    for cam_idx in range(graph.ncams_):
        P = graph.intrinsics_mat_[cam_idx] @ graph.extrinsics_mat_[cam_idx]
        pts3D_h = np.hstack((graph.structure_points_, np.ones((len(graph.structure_points_), 1))))
        proj = (P @ pts3D_h.T).T
        proj = proj[:, :2] / proj[:, 2, np.newaxis]

        # Assuming each track stores the corresponding observed 2D points
        observed = np.array([t.pt1 if cam_idx == 0 else t.pt2 for t in graph.tracks_])
        err = np.linalg.norm(proj - observed, axis=1)

        total_error += np.sum(err)
        total_points += len(err)

    return total_error / max(total_points, 1)


# Example usage:
triangulate_nonlinear(graph[i])
error = reprojection_error(graph[i])
print(f"Reprojection error (before bundle adjustment): {error:.4f}")


In [ ]:
import numpy as np
import cv2
from scipy.optimize import least_squares

def Open3DCVBundleAdjustment(graph, mode="BUNDLE_NO_INTRINSICS"):
    """
    Simplified Python equivalent of Open3DCVBundleAdjustment.
    Supports:
      - "BUNDLE_NO_INTRINSICS": only refines poses and 3D points
      - "BUNDLE_FOCAL_LENGTH": refines focal length too
      - "BUNDLE_INTRINSICS": refines full intrinsics
    """

    # Flatten camera parameters (R,t) for optimization
    camera_params = []
    for P in graph.extrinsics_mat_:
        R = P[:, :3]
        t = P[:, 3]
        rvec, _ = cv2.Rodrigues(R)
        camera_params.append(np.hstack((rvec.flatten(), t.flatten())))
    camera_params = np.concatenate(camera_params)

    points_3d = graph.structure_points_.copy()

    # Bundle intrinsics
    f = graph.intrinsics_mat_[0][0, 0]
    cx, cy = graph.intrinsics_mat_[0][0, 2], graph.intrinsics_mat_[0][1, 2]

    # Prepare observations
    obs = []
    for cam_idx in range(graph.ncams_):
        obs.append(np.array([t.pt1 if cam_idx == 0 else t.pt2 for t in graph.tracks_]))
    obs = np.stack(obs, axis=0)  # shape (n_cams, n_points, 2)

    def project(points, rvec, tvec, f, cx, cy):
        pts, _ = cv2.projectPoints(points, rvec, tvec, np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]]), None)
        return pts.reshape(-1, 2)

    def residual(params):
        # Unpack parameters
        ptr = 0
        res = []
        for cam_idx in range(graph.ncams_):
            rvec = params[ptr:ptr+3]
            tvec = params[ptr+3:ptr+6]
            ptr += 6
            proj = project(points_3d, rvec, tvec, f, cx, cy)
            res.append((proj - obs[cam_idx]).ravel())
        return np.concatenate(res)

    # Run nonlinear least squares optimization
    least_squares(residual, camera_params, verbose=1, ftol=1e-8, xtol=1e-8, method="lm")

    # In real implementations, the optimized parameters would update graph.extrinsics_mat_ and intr


In [ ]:
import numpy as np
import cv2
from scipy.optimize import least_squares
from PIL import Image, ExifTags  # optional, for EXIF focal retrieval
import math
import copy

# ---------- Helper utilities ----------

def get_focal_from_exif(image_path):
    """Try to extract focal length in pixels from EXIF. Return None if unavailable."""
    try:
        im = Image.open(image_path)
        exif = im._getexif()
        if not exif:
            return None
        # find focal length tag id
        for tag, name in ExifTags.TAGS.items():
            if name == 'FocalLength':
                focal_tag = tag
                break
        if focal_tag not in exif:
            return None
        fl = exif[focal_tag]  # usually a tuple (num, den)
        if isinstance(fl, tuple):
            fl_value = float(fl[0]) / float(fl[1])
        else:
            fl_value = float(fl)
        # If pixel size / sensor size unknown, this is focal in mm. We can't convert reliably
        return fl_value
    except Exception:
        return None

def build_mat_from_params(f):
    """Return K given focal length f (float) and assumed principal point at image center set later."""
    return np.array([[f, 0.0, 0.0],
                     [0.0, f, 0.0],
                     [0.0, 0.0, 1.0]], dtype=float)

def baseline_angle_from_extrinsics(M1, M2):
    """Compute angle between camera optical axes using extrinsic matrices: M = [R|t]."""
    R1 = M1[:, :3]
    R2 = M2[:, :3]
    # camera optical axis is the third column of R^T (i.e., R[:,2])
    z1 = R1[:, 2]
    z2 = R2[:, 2]
    # angle
    cosang = np.clip(np.dot(z1, z2) / (np.linalg.norm(z1) * np.linalg.norm(z2)), -1.0, 1.0)
    return math.degrees(math.acos(cosang))

# ---------- Core pipeline functions ----------

def estimate_fundamental_and_inliers(pair, keys, ransac_reproj_thresh=1.0, confidence=0.99):
    """
    Build point correspondences from pair.matches_, estimate F using RANSAC,
    return F (3x3) and inlier_mask (boolean array).
    """
    ind1, ind2 = pair.cams_
    if len(pair.matches_) < 8:
        return None, None

    pts1 = []
    pts2 = []
    for m in pair.matches_:
        kp1 = keys[ind1][m.ind_key_[0]].pt
        kp2 = keys[ind2][m.ind_key_[1]].pt
        pts1.append(kp1)
        pts2.append(kp2)

    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)

    # use RANSAC to estimate fundamental matrix
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, ransac_reproj_thresh, confidence)
    if F is None or mask is None:
        return None, None
    mask = mask.ravel().astype(bool)
    return F, mask

def essential_from_F(F, K1, K2):
    """Compute E = K2^T * F * K1"""
    return K2.T @ F @ K1

def recover_pose_from_E(E, pts1, pts2, K1, K2):
    """Use cv2.recoverPose to obtain R, t and mask of inliers."""
    # OpenCV expects pts in shape Nx2 and type float
    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)
    # normalize by K (recoverPose accepts K and pts in pixel coords)
    retval, R, t, mask = cv2.recoverPose(E, pts1, pts2, K1)
    mask = mask.ravel().astype(bool)
    return R, t, mask

def triangulate_pair(K1, M1, K2, M2, pts1, pts2):
    """
    Linear triangulation via cv2.triangulatePoints.
    M1, M2 are 3x4 extrinsic matrices.
    pts1, pts2 Nx2 (float32)
    Returns Nx3 array of 3D points.
    """
    P1 = K1 @ M1
    P2 = K2 @ M2
    pts1_t = np.array(pts1, dtype=np.float64).T
    pts2_t = np.array(pts2, dtype=np.float64).T
    points4d = cv2.triangulatePoints(P1, P2, pts1_t, pts2_t)  # 4xN
    points3d = (points4d[:3] / points4d[3]).T
    return points3d

def simple_bundle_adjust(graph, optimize_focal=False, optimize_intrinsics=False, max_nfev=200):
    """
    Lightweight bundle adjustment that refines camera poses and optionally focal length.
    This is a simplified version: assumes small graph (2 views) and that correspondences are aligned.
    """
    # Flatten camera parameters (rvec, tvec) and optionally focal length
    n_cams = graph.ncams_
    n_points = len(graph.structure_points_)
    if n_cams < 2 or n_points == 0:
        return

    # pack camera params
    cam_params = []
    for M in graph.extrinsics_mat_:
        R = M[:, :3]
        t = M[:, 3]
        rvec, _ = cv2.Rodrigues(R)
        cam_params.append(np.hstack((rvec.flatten(), t.flatten())))
    cam_params = np.hstack(cam_params)  # length 6*n_cams

    points3d = graph.structure_points_.copy().reshape(-1)  # 3*n_points

    # intrinsics (we assume same K for all cams here for simplicity)
    K0 = graph.intrinsics_mat_[0]
    f0 = K0[0,0]
    cx = K0[0,2]
    cy = K0[1,2]

    # build observed 2D array obs[cam_idx] = Nx2
    obs = []
    for cam_idx in range(n_cams):
        cam_obs = []
        for tr in graph.tracks_:
            # track expected to contain per-camera 2D observation: track.obs[cam_idx] or approximated as pt1/pt2
            if cam_idx == 0:
                cam_obs.append(tr.pt1)
            else:
                cam_obs.append(tr.pt2)
        obs.append(np.array(cam_obs, dtype=np.float64))
    obs = np.array(obs)  # shape (n_cams, n_points, 2)

    def project_point(p3, rvec, tvec, f):
        pts, _ = cv2.projectPoints(p3[np.newaxis,:], rvec, tvec, np.array([[f, 0, cx],[0,f,cy],[0,0,1]]), None)
        return pts.ravel()

    def residual(x):
        # x contains cam_params and points3d and maybe f
        ptr = 0
        camp = []
        for _ in range(n_cams):
            r = x[ptr:ptr+3]; t = x[ptr+3:ptr+6]; ptr += 6
            camp.append((r, t))
        pts = x[ptr:ptr+3*n_points].reshape((n_points, 3))
        ptr += 3*n_points
        if optimize_focal:
            f = x[ptr]
        else:
            f = f0

        res = []
        for ci in range(n_cams):
            rvec, tvec = camp[ci]
            for pi in range(n_points):
                proj = project_point(pts[pi], rvec, tvec, f)  # returns (x,y)
                res.extend(proj - obs[ci, pi])
        return np.array(res)

    # initial x
    x0 = np.hstack([cam_params, points3d])
    if optimize_focal:
        x0 = np.hstack([x0, f0])

    result = least_squares(residual, x0, verbose=0, max_nfev=max_nfev, xtol=1e-8, ftol=1e-8, method='lm')
    # Update graph from result (very minimal)
    # Unpack updated camera params and points
    x = result.x
    ptr = 0
    for idx in range(n_cams):
        rvec = x[ptr:ptr+3]; tvec = x[ptr+3:ptr+6]; ptr += 6
        R, _ = cv2.Rodrigues(rvec)
        t = tvec.reshape(3,1)
        graph.extrinsics_mat_[idx] = np.hstack((R, t))
    pts = x[ptr:ptr+3*n_points].reshape((n_points, 3))
    graph.structure_points_ = pts
    ptr += 3*n_points
    if optimize_focal and ptr < len(x):
        f_opt = x[ptr]
        # update intrinsics for all cameras
        for i in range(len(graph.intrinsics_mat_)):
            K = graph.intrinsics_mat_[i]
            K[0,0] = K[1,1] = f_opt
            graph.intrinsics_mat_[i] = K

def verify_graph(graph, min_baseline_angle=1.0, max_reproj=5.0):
    """Check baseline angle and reprojection error to accept/reject the graph."""
    if graph.ncams_ < 2:
        return False
    # baseline angle
    ang = baseline_angle_from_extrinsics(graph.extrinsics_mat_[0], graph.extrinsics_mat_[1])
    err = reprojection_error(graph)
    ok = (abs(ang) >= min_baseline_angle) and (err <= max_reproj)
    return ok, ang, err

# ---------- Main loop over pairs: converts pseudocode to python ----------

def two_view_sfm_main(pairs, keys, images, global_odir=None,
                      thresh_inlier_ratio=0.2, update_focal=False, update_intrinsic=False):
    """
    Main loop:
      For each pair in pairs:
        - robustly estimate F/E
        - estimate pose
        - triangulate points
        - bundle adjust
        - verify checks
        - construct Graph g from pair p and append to list if successful
    Returns list of Graph objects built successfully.
    """
    graphs = []

    for p_idx, pair in enumerate(pairs):
        print("*******************************")
        print(f" 2-View SfM of image {pair.cams_[0]+1} and {pair.cams_[1]+1}")
        print("*******************************")

        # 1) Estimate fundamental matrix and inlier mask
        F, inlier_mask = estimate_fundamental_and_inliers(pair, keys)
        if F is None:
            print("F estimation failed, skipping pair.")
            continue
        inlier_ratio = np.sum(inlier_mask) / len(inlier_mask)
        print(f"ratio of matching inliers: {inlier_ratio:.3f}")
        if inlier_ratio < thresh_inlier_ratio:
            print("Too few inliers, skip.")
            continue

        # update matches to inliers only
        pair.update_matches(inlier_mask)

        # 2) intrinsics: try to get from pair or EXIF (we need width/height and focal)
        # If pair.intrinsics_mat_ not set, use fallback focal and image size
        w = images[pair.cams_[0]].shape[1]
        h = images[pair.cams_[0]].shape[0]
        # If you have EXIF path info, try to read focal in mm (optional)
        # For now use the hard-coded focal from your earlier snippet if desired, else approximate
        # Example fallback focal in pixels (you can tune)
        fallback_f = 1520.4
        pair.update_intrinsics(fallback_f, w, h)
        K1 = pair.intrinsics_mat_[0]
        K2 = pair.intrinsics_mat_[1]

        # 3) Compute Essential matrix and recover pose
        pair.E_ = essential_from_F(F, K1, K2)

        # prepare pts for recoverPose (only inlier points)
        ind1, ind2 = pair.cams_
        pts1 = []
        pts2 = []
        for m in pair.matches_:
            pts1.append(keys[ind1][m.ind_key_[0]].pt)
            pts2.append(keys[ind2][m.ind_key_[1]].pt)

        R, t, pose_mask = recover_pose_from_E(pair.E_, pts1, pts2, K1, K2)

        # store extrinsics into pair (camera1 at origin)
        M1 = np.hstack


In [ ]:
import numpy as np
import cv2
from scipy.optimize import least_squares
import math
import copy
from typing import List, Tuple, Dict, Optional

# ---------------------
# Utility helpers used below (if you already have these from previous code, use those)
# ---------------------
def project_point_with_P(P: np.ndarray, X: np.ndarray) -> np.ndarray:
    """Project a 3D point X (3,) with 3x4 P, return 2D (x,y)."""
    Xh = np.array([X[0], X[1], X[2], 1.0], dtype=float)
    p = P @ Xh
    return (p[:2] / p[2]).astype(float)

def triangulate_point_multiview(Ps: List[np.ndarray], pts: List[Tuple[float, float]]) -> Optional[np.ndarray]:
    """
    Triangulate a single 3D point from multiple views.
    Ps: list of 3x4 projection matrices
    pts: list of corresponding 2D points (x,y)
    returns 3-vector or None if failure
    """
    if len(Ps) < 2 or len(Ps) != len(pts):
        return None
    A_rows = []
    for P, (x, y) in zip(Ps, pts):
        # x * P[2,:] - P[0,:]
        A_rows.append(x * P[2, :] - P[0, :])
        A_rows.append(y * P[2, :] - P[1, :])
    A = np.vstack(A_rows)  # (2n x 4)
    # Solve by SVD
    try:
        _, _, Vt = np.linalg.svd(A)
        Xh = Vt[-1, :]
        if abs(Xh[3]) < 1e-12:
            return None
        X = Xh[:3] / Xh[3]
        return X
    except np.linalg.LinAlgError:
        return None

def compute_point_reprojection_error_for_track(X: np.ndarray, graph: "Graph", track, return_per_view=False):
    """
    For a given track (with per-camera 2D observations stored as track.obs: dict cam_idx->(x,y) or attributes pt1/pt2 for two-view),
    compute per-view residuals and mean.
    """
    residuals = []
    for cam_idx, P in enumerate(graph.extrinsics_mat_):
        # Need K to form P_camera = K @ [R|t] or we can assume graph.intrinsics_mat_ holds Ks aligned to graph.ind_cam_
        # We'll assume graph.intrinsics_mat_[cam_idx] corresponds to graph.extrinsics_mat_[cam_idx]
        if hasattr(track, "obs") and cam_idx in track.obs:
            K = graph.intrinsics_mat_[cam_idx]
            M = graph.extrinsics_mat_[cam_idx]
            Pcam = K @ M
            proj = project_point_with_P(Pcam, X)
            observed = np.array(track.obs[cam_idx], dtype=float)
            residuals.append(np.linalg.norm(proj - observed))
    if len(residuals) == 0:
        return (np.inf if not return_per_view else ([], np.inf))
    if return_per_view:
        return residuals, float(np.mean(residuals))
    return float(np.mean(residuals))

def baseline_angle_for_track(track, graph: "Graph"):
    """
    Compute maximum baseline angle between camera optical axes that observe this track.
    track.obs is expected to contain camera indices -> 2D points.
    Returns angle in degrees.
    """
    cams = sorted(list(track.obs.keys()))
    if len(cams) < 2:
        return 0.0
    axes = []
    for ci in cams:
        M = graph.extrinsics_mat_[ci]
        R = M[:, :3]
        z = R[:, 2]  # optical axis direction in world coords (camera forwards)
        axes.append(z / np.linalg.norm(z))
    # compute max angle between any pair
    max_ang = 0.0
    for i in range(len(axes)):
        for j in range(i + 1, len(axes)):
            dot = np.clip(np.dot(axes[i], axes[j]), -1.0, 1.0)
            ang = math.degrees(math.acos(dot))
            if ang > max_ang:
                max_ang = ang
    return max_ang

# ---------------------
# Core merge & refine pipeline
# ---------------------

def pick_best_graph_to_merge(global_graph: "Graph", graphs: List["Graph"]) -> Optional[int]:
    """
    Pick the index of the graph in 'graphs' that has the maximum number of common track ids with global_graph.
    Assumes tracks have an 'id' attribute.
    Returns index or None if none share tracks.
    """
    global_ids = {t.id for t in global_graph.tracks_}
    best_idx = None
    best_common = 0
    for idx, g in enumerate(graphs):
        ids = {t.id for t in g.tracks_}
        common = len(global_ids.intersection(ids))
        if common > best_common:
            best_common = common
            best_idx = idx
    return best_idx

def merge_graph_into_global(global_graph: "Graph", g: "Graph") -> "Graph":
    """
    Merge graph g into global_graph. This does:
      - merges camera lists (appends new cams, tracks),
      - merges intrinsics/extrinsics aligned with cameras,
      - merges tracks by track.id (common tracks are unified; new tracks appended).
    Returns updated global_graph (modified in place as well).
    """
    # map from incoming camera id to index in global_graph
    # We'll assume both graphs use camera identifiers in ind_cam_ which are unique (e.g. original image indices).
    # If not, we use their ind_cam_ lists directly.
    # Build mapping: g.ind_cam_ -> global index (append if new)
    cam_map = {}
    for i, camid in enumerate(g.ind_cam_):
        if camid in global_graph.ind_cam_:
            cam_map[i] = global_graph.ind_cam_.index(camid)
        else:
            # append camera to global graph
            global_graph.ind_cam_.append(camid)
            global_graph.intrinsics_mat_.append(g.intrinsics_mat_[i].copy())
            global_graph.extrinsics_mat_.append(g.extrinsics_mat_[i].copy())
            cam_map[i] = len(global_graph.ind_cam_) - 1
            global_graph.ncams_ += 1

    # Merge tracks by id
    # Build quick lookup for global tracks
    global_tracks_by_id = {t.id: t for t in global_graph.tracks_}

    for tr in g.tracks_:
        tid = tr.id
        # ensure track.obs exists: obs: dict cam_idx -> (x,y). If not, create it from pt1/pt2 for two-view case.
        if not hasattr(tr, "obs"):
            tr.obs = {}
            # guess two-view style pt1/pt2 aligned to g.ind_cam_
            if hasattr(tr, "pt1") and hasattr(tr, "pt2") and len(g.ind_cam_) >= 2:
                tr.obs[g.ind_cam_[0]] = tuple(np.array(tr.pt1).astype(float))
                tr.obs[g.ind_cam_[1]] = tuple(np.array(tr.pt2).astype(float))

        if tid in global_tracks_by_id:
            # merge observations: add any observations from tr into existing global track
            gtr = global_tracks_by_id[tid]
            if not hasattr(gtr, "obs"):
                gtr.obs = {}
            for cam_local_idx, obs_xy in tr.obs.items():
                # map cam_local_idx in g to global index via cam_map and then to the camera id
                # But tr.obs keys are camera **ids** or local indices? We wrote obs keyed by camera id above.
                # assume keys are camera ids (consistent with tr.id being global id).
                camid = cam_local_idx
                # find if camid is in global_graph.ind_cam_
                if camid in global_graph.ind_cam_:
                    gtr.obs[camid] = tuple(np.array(obs_xy, dtype=float))
                else:
                    # improbable, but if tr.obs keyed by local indices, map them
                    # if cam_local_idx < len(g.ind_cam_): map using cam_map
                    if isinstance(cam_local_idx, int) and cam_local_idx in cam_map:
                        mapped_idx = cam_map[cam_local_idx]
                        mapped_camid = global_graph.ind_cam_[mapped_idx]
                        gtr.obs[mapped_camid] = tuple(np.array(obs_xy, dtype=float))
                    else:
                        # skip ambiguous observation
                        continue
        else:
            # new track, remap its obs keys to global camera ids and append
            new_tr = copy.deepcopy(tr)
            if not hasattr(new_tr, "obs"):
                new_tr.obs = {}
                if hasattr(new_tr, "pt1") and hasattr(new_tr, "pt2") and len(g.ind_cam_) >= 2:
                    new_tr.obs[g.ind_cam_[0]] = tuple(np.array(new_tr.pt1).astype(float))
                    new_tr.obs[g.ind_cam_[1]] = tuple(np.array(new_tr.pt2).astype(float))
            # remap keys if needed
            remapped_obs = {}
            for key, val in new_tr.obs.items():
                if key in g.ind_cam_:
                    local_idx = g.ind_cam_.index(key)
                    mapped_global_idx = cam_map[local_idx]
                    mapped_camid = global_graph.ind_cam_[mapped_global_idx]
                    remapped_obs[mapped_camid] = tuple(np.array(val, dtype=float))
                else:
                    # if key is already a global cam id (likely), keep it
                    remapped_obs[key] = tuple(np.array(val, dtype=float))
            new_tr.obs = remapped_obs
            global_graph.tracks_.append(new_tr)

    # Done merging cameras and tracks
    return global_graph

def triangulate_all_tracks_in_graph(graph: "Graph", min_views=2):
    """
    For every track in graph.tracks_, triangulate from all available views and set graph.structure_points_.
    Only triangulates tracks observed in >= min_views.
    """
    pts3d = []
    valid_tracks = []
    for tr in graph.tracks_:
        if not hasattr(tr, "obs"):
            continue
        cams = sorted(list(tr.obs.keys()))
        if len(cams) < min_views:
            continue
        Ps = []
        pts = []
        for camid in cams:
            # get index of camid in graph.ind_cam_
            if camid not in graph.ind_cam_:
                continue
            idx = graph.ind_cam_.index(camid)
            K = graph.intrinsics_mat_[idx]
            M = graph.extrinsics_mat_[idx]
            Pcam = K @ M
            Ps.append(Pcam)
            pts.append(tuple(map(float, tr.obs[camid])))
        if len(Ps) < min_views:
            continue
        X = triangulate_point_multiview(Ps, pts)
        if X is not None and np.isfinite(X).all():
            pts3d.append(X)
            valid_tracks.append(tr)
    graph.structure_points_ = np.array(pts3d, dtype=float) if len(pts3d) > 0 else np.zeros((0,3))
    # Optionally keep only the valid_tracks in graph.tracks_
    graph.tracks_ = valid_tracks

def remove_outlier_tracks(graph: "Graph", min_baseline_angle_deg=1.0, max_reproj_error=5.0):
    """
    Remove track entries (and associated 3D points) that fail baseline angle or reprojection error tests.
    Assumes graph.structure_points_ aligns index-wise with graph.tracks_.
    """
    if graph.structure_points_ is None or len(graph.structure_points_) == 0:
        return
    good_pts = []
    good_tracks = []
    for idx, tr in enumerate(graph.tracks_):
        X = graph.structure_points_[idx]
        base_ang = baseline_angle_for_track(tr, graph)
        _, reproj_err = compute_point_reprojection_error_for_track(X, graph, tr, return_per_view=True)
        mean_reproj = reproj_err
        if base_ang >= min_baseline_angle_deg and mean_reproj <= max_reproj_error:
            good_pts.append(X)
            good_tracks.append(tr)
    graph.structure_points_ = np.array(good_pts, dtype=float) if len(good_pts) > 0 else np.zeros((0,3))
    graph.tracks_ = good_tracks

# A minimal reprojection_error function for Graph using matched ordering:
def reprojection_error_graph(graph: "Graph") -> float:
    """
    compute mean reprojection error across all tracks and views present in graph.
    """
    if graph.structure_points_ is None or len(graph.structure_points_) == 0:
        return np.inf
    total_err = 0.0
    count = 0
    for idx, tr in enumerate(graph.tracks_):
        X = graph.structure_points_[idx]
        for camid, obs in tr.obs.items():
            if camid not in graph.ind_cam_:
                continue
            cam_idx = graph.ind_cam_.index(camid)
            K = graph.intrinsics_mat_[cam_idx]
            M = graph.extrinsics_mat_[cam_idx]
            Pcam = K @ M
            proj = project_point_with_P(Pcam, X)
            total_err += np.linalg.norm(proj - np.array(obs))
            count += 1
    return (total_err / count) if count > 0 else np.inf

# A stub/simple BA wrapper that calls simple_bundle_adjust you already have (adapts signatures)
def run_bundle_adjustment_graph(graph: "Graph", optimize_focal=False, optimize_intrinsics=False):
    """
    Wraps the previously defined simple_bundle_adjust function (which expects graph fields
    similarly named). Here we assume graph.extrinsics_mat_ is a list of 3x4 matrices and
    graph.intrinsics_mat_ a list of 3x3 Ks.
    """
    # Reuse simple_bundle_adjust from earlier code. If you haven't defined it exactly,
    # replace with a call to your own bundle adjustment implementation.
    simple_bundle_adjust(graph, optimize_focal=optimize_focal, optimize_intrinsics=optimize_intrinsics)

# ---------------------
# Top-level merging loop
# ---------------------

def merge_graphs_into_global(graphs: List["Graph"],
                             global_graph: "Graph",
                             min_common_tracks = 2,
                             min_views_for_triangulation = 2,
                             min_baseline_angle_deg = 1.0,
                             max_reproj_error = 5.0,
                             do_second_ba = True,
                             update_focal=False,
                             update_intrinsics=False):
    """
    Merge graphs in 'graphs' into global_graph iteratively using your pseudocode:
      - pick graph g with max common tracks
      - merge
      - triangulate
      - BA
      - remove track outliers by baseline angle and reprojection error
      - BA
    Returns final global_graph.
    """
    remaining = graphs.copy()
    while remaining:
        best_idx = pick_best_graph_to_merge(global_graph, remaining)
        if best_idx is None:
            # pick any with at least min_common_tracks? or break
            # If no graphs share tracks, we can optionally pick the next largest or break.
            print("No remaining graph shares tracks with global_graph. Stopping merge.")
            break
        g = remaining.pop(best_idx)
        print(f"Merging graph with cameras {g.ind_cam_} into global graph (global cams: {global_graph.ind_cam_})")

        # quick check: require at least min_common_tracks in common (adjust policy)
        common_ids = set(t.id for t in global_graph.tracks_).intersection({t.id for t in g.tracks_})
        if len(common_ids) < min_common_tracks:
            print(f"Graph has only {len(common_ids)} common tracks (< {min_common_tracks}), skipping.")
            continue

        # 1) merge
        merge_graph_into_global(global_graph, g)

        # 2) triangulate tracks in merged global graph
        triangulate_all_tracks_in_graph(global_graph, min_views=min_views_for_triangulation)

        # 3) bundle adjustment (coarse)
        print("Running bundle adjustment (coarse)...")
        run_bundle_adjustment_graph(global_graph, optimize_focal=update_focal, optimize_intrinsics=update_intrinsics)

        # 4) remove outliers (baseline angle + reprojection)
        print("Filtering outlier tracks...")
        remove_outlier_tracks(global_graph, min_baseline_angle_deg=min_baseline_angle_deg, max_reproj_error=max_reproj_error)

        # 5) optional second BA
        if do_second_ba:
            print("Running bundle adjustment (final)...")
            run_bundle_adjustment_graph(global_graph, optimize_focal=update_focal, optimize_intrinsics=update_intrinsics)

        # report
        err = reprojection_error_graph(global_graph)
        print(f"Global graph now has {len(global_graph.tracks_)} tracks, reproj error: {err:.4f}")

    return global_graph

# ---------------------
# Example usage:
# ---------------------
# global_graph = Graph()  # initial global graph created from one seed pair/graph
# final_global = merge_graphs_into_global(graphs, global_graph, min_common_tracks=3)


In [ ]:
print("=======================================")
print("   Multi-view Incremental SfM Process   ")
print("=======================================")

# Initialize global graph with the first pair
global_graph = Graph(graphs[0])

for i in range(1, nimages - 1):
    print("*******************************")
    print(f" N-View SfM: merging graph 0-{i}")
    print("*******************************")

    # ------ merge graphs ------
    Graph.merge_graph(global_graph, graphs[i])

    # ------ N-view triangulation ------
    triangulate_nonlinear(global_graph)
    error = reprojection_error(global_graph)
    print(f"Reprojection error (before bundle adjustment): {error:.4f}")

    # ------ N-view bundle adjustment ------
    print("------ start bundle adjustment ------")
    Open3DCVBundleAdjustment(global_graph, "BUNDLE_PRINCIPAL_POINT")
    print("------ end bundle adjustment ------")

    # ------ Evaluate error ------
    error = reprojection_error(global_graph)
    print(f"Reprojection error (after bundle adjustment): {error:.4f}")


In [ ]:
def inv_Rt(Rt):
    """Inverse of a [3x4] pose matrix."""
    R = Rt[:, :3]
    t = Rt[:, 3]
    R_inv = R.T
    t_inv = -R_inv @ t
    Rt_inv = np.hstack((R_inv, t_inv.reshape(3, 1)))
    return Rt_inv


def concat_Rt(Rt1, Rt2):
    """Compose two extrinsic matrices: Rt_result = Rt1 * Rt2"""
    R1, t1 = Rt1[:, :3], Rt1[:, 3]
    R2, t2 = Rt2[:, :3], Rt2[:, 3]
    R = R1 @ R2
    t = R1 @ t2 + t1
    return np.hstack((R, t.reshape(3, 1)))


In [ ]:
import numpy as np
from bisect import bisect_left

def merge_graph(graph1, graph2):
    """
    Merge graph2 into graph1.
    Equivalent to: void Graph::merge_graph(Graph &graph1, Graph &graph2)
    """

    # ----- find overlapping cameras -----
    cams_common = sorted(list(set(graph1.ind_cam_) & set(graph2.ind_cam_)))
    cams_diff = sorted(list(set(graph2.ind_cam_) - set(graph1.ind_cam_)))

    if not cams_common:
        return  # nothing to merge if no shared cameras

    # ----- merge camera intrinsics & extrinsics -----
    ind_cam1 = graph1.index(cams_common[0])
    ind_cam2 = graph2.index(cams_common[0])

    Rt1 = graph1.extrinsics_mat_[ind_cam1]
    Rt2 = graph2.extrinsics_mat_[ind_cam2]

    # Rt21 = concat_Rt(inv_Rt(Rt1), Rt2)
    Rt21 = concat_Rt(inv_Rt(Rt1), Rt2)

    # transform structure points of graph2 into graph1 coordinate frame
    for pt in graph2.structure_points_:
        pt_h = np.append(pt.coords(), 1.0)
        pt.coords_ = (Rt21 @ pt_h)[:3]

    Rt21t = inv_Rt(Rt21)

    # append non-overlapping cameras
    for cam_id in cams_diff:
        graph1.ind_cam_.append(cam_id)
        graph1.intrinsics_mat_.append(graph2.intrinsics_mat_[graph2.index(cam_id)])
        Rt2_rel = concat_Rt(graph2.extrinsics_mat_[graph2.index(cam_id)], Rt21t)
        graph1.extrinsics_mat_.append(Rt2_rel)

    # sort cameras (keep consistent ordering)
    order = np.argsort(graph1.ind_cam_)
    graph1.ind_cam_ = [graph1.ind_cam_[i] for i in order]
    graph1.intrinsics_mat_ = [graph1.intrinsics_mat_[i] for i in order]
    graph1.extrinsics_mat_ = [graph1.extrinsics_mat_[i] for i in order]
    graph1.ncams_ = len(graph1.ind_cam_)

    # ----- merge feature tracks -----
    ntracks = len(graph1.tracks_)
    for track2, struct_pt in zip(graph2.tracks_, graph2.structure_points_):
        is_connected = False
        for i in range(ntracks):
            track1 = graph1.tracks_[i]
            feats_common = Track.find_overlapping_keypoints(track1, track2)
            if feats_common:
                Graph.merge_tracks(track1, track2, feats_common)
                is_connected = True
                break
        if not is_connected:
            graph1.add_track(track2)
            graph1.add_struct_pt(struct_pt)

    # new features from non-overlapping cameras skipped for now
